<a href="https://colab.research.google.com/github/duonghung86/Injury-severity-classification/blob/main/VCA_2_1_MLP_resampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from psutil import virtual_memory,cpu_percent
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print('Current system-wide CPU utilization %: ',cpu_percent())
#Remove all warning
import warnings
warnings.filterwarnings("ignore")

Your runtime has 270.0 gigabytes of available RAM

Current system-wide CPU utilization %:  0.0


In [2]:
# Basic packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
import os
# Preprocessing
from sklearn.preprocessing import StandardScaler # Standardization
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Machine learning algos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from collections import Counter
from imblearn.over_sampling import SMOTE, RandomOverSampler,BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler,NearMiss,EditedNearestNeighbours
# Metrics
from imblearn.metrics import geometric_mean_score
# Tensorflow
import tensorflow as tf
print(tf.__version__)
from tensorflow import feature_column  # for data wrangling
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_addons.metrics import CohenKappa,F1Score
from tensorboard.plugins.hparams import api as hp

2.1.0


# Load data

In [3]:
url = 'https://github.com/duonghung86/Injury-severity-classification/blob/main/Prepared%20Texas%202019.zip?raw=true' 
data_path = tf.keras.utils.get_file(origin=url, fname=url.split('/')[-1].split('?')[0], extract=True)
data_path = data_path.replace('%20',' ').replace('.zip','.csv')

In [4]:
# Load data
df = pd.read_csv(data_path)
print(df.shape)
df.head(3)

(949856, 19)


,Prsn_Injry_Sev,Prsn_Age,Prsn_Gndr,Wthr_Cond,Light_Cond,Surf_Cond,Veh_Body_Styl,Prsn_Rest,Prsn_Drg_Rslt,Harm_Evnt,Rural,Crash_Speed_Limit,Road_Algn,Veh_Mod_Year,Weekend,Crash_season,Part_of_day,Collsn_type,Collsn_name
0,0,26,MALE,CLEAR,DAYLIGHT,DRY,"PASSENGER CAR, 2-DOOR",SHOULDER & LAP BELT,Not Applicable,MOTOR VEHICLE IN TRANSPORT,0,-1,"STRAIGHT, LEVEL",33,1,3,3,SAME DIRECTION,ONE STRAIGHT-ONE LEFT TURN
1,0,52,MALE,CLEAR,DAYLIGHT,DRY,"PASSENGER CAR, 2-DOOR",SHOULDER & LAP BELT,Not Applicable,MOTOR VEHICLE IN TRANSPORT,0,-1,"STRAIGHT, LEVEL",19,1,3,3,SAME DIRECTION,ONE STRAIGHT-ONE LEFT TURN
2,0,27,FEMALE,CLEAR,DAYLIGHT,DRY,PICKUP,SHOULDER & LAP BELT,Not Applicable,MOTOR VEHICLE IN TRANSPORT,1,-1,"CURVE, LEVEL",16,1,3,4,SAME DIRECTION,BOTH LEFT TURN


In [5]:
# Let's just use 80% of the total dataset
df, _ = train_test_split(df, test_size=0.80,stratify = df['Prsn_Injry_Sev'])
df.shape

(189971, 19)

In [6]:
y = df['Prsn_Injry_Sev']
print('All target values:')
print(y.value_counts())
X = df.drop(columns=['Prsn_Injry_Sev'])

All target values:
0    158512
1     20482
2      9048
3      1590
4       339
Name: Prsn_Injry_Sev, dtype: int64


In [7]:
# %% Data wrangling -------------
# Classify variable type
emb_vars, ind_vars, num_vars = [], [], []
for var in X.columns:
    if X[var].dtypes == 'O':
        if len(X[var].unique()) > 5:
            emb_vars.append(var)
        else:
            ind_vars.append(var)
    else:
        num_vars.append(var)
print('Numerical variables are ', num_vars)
print('Categorical variables that have at most 5 categories are ', ind_vars)
print('Categorical variables that have more than 5 categories are ', emb_vars)

# Create feature columns
feature_columns = []
# numeric cols
for header in num_vars:
    feature_columns.append(feature_column.numeric_column(header))
# bucketized cols
# age = feature_column.numeric_column('Prsn_Age')
# age_buckets = feature_column.bucketized_column(age, boundaries=[16, 22, 35, 55, 65])
# feature_columns.append(age_buckets)
# indicator_columns
for col_name in ind_vars:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
        col_name, X[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)
# embedding columns
for col_name in emb_vars:
    emb_column = feature_column.categorical_column_with_vocabulary_list(
        col_name, X[col_name].unique())
    col_embedding = feature_column.embedding_column(emb_column, dimension=5)
    feature_columns.append(col_embedding)

# Convert all setup into new dataset
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
X = feature_layer(dict(X)).numpy()
print('New shape of the input data set:',X.shape)

Numerical variables are  ['Prsn_Age', 'Rural', 'Crash_Speed_Limit', 'Veh_Mod_Year', 'Weekend', 'Crash_season', 'Part_of_day']
Categorical variables that have at most 5 categories are  ['Prsn_Gndr', 'Prsn_Drg_Rslt', 'Collsn_type']
Categorical variables that have more than 5 categories are  ['Wthr_Cond', 'Light_Cond', 'Surf_Cond', 'Veh_Body_Styl', 'Prsn_Rest', 'Harm_Evnt', 'Road_Algn', 'Collsn_name']
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
New shape of the input data set: (189971, 59)


In [8]:
# %% Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=48)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=48)

print('Training features shape:', X_train.shape)
print('Validation features shape:', X_val.shape)
print('Test features shape:', X_test.shape)

# %% standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

Training features shape: (121580, 59)
Validation features shape: (30396, 59)
Test features shape: (37995, 59)


In [9]:
# %% Function to compare the prediction and true labels
def get_accs(label, prediction, tr_time,  show=False):
    cm = confusion_matrix(label, prediction)
    length = cm.shape[0]
    num_cases = len(label)
    # global accuracy
    glb_acc = np.trace(cm) / len(label)
    ind_accs = cm / np.sum(cm, axis=1)[:, np.newaxis]
    accs = [ind_accs[i, i] for i in range(length)]
    index = ['Class {}'.format(i) for i in range(length)]
    # Global accuracy
    accs.append(glb_acc)
    #index.append
    # G-mean
    accs.append(geometric_mean_score(label, prediction, correction=0.001))
    #index.append('G-mean')
    # Average perf
    accs.append((glb_acc + accs[-1]) / 2)
    #index.append('Avg_Pfm')
    # Training time
    accs.append(np.round(tr_time,3))
    index = index + ['Overall Accuracy','G-mean','Avg_Pfm','Training Time']
    # Plot confusion matrix
    plot_dict = {'Confusion matrix': (cm,'g'),
                 'Normalized confusion matrix': (ind_accs,'.2f')}
    if show:
        plt.figure(figsize=(14, 6))
        i = 1
        for key, value in plot_dict.items():
            plt.subplot(1, 2, i)
            sns.heatmap(value[0], xticklabels=np.arange(length), yticklabels=np.arange(length),
                        annot=True, fmt=value[1], cmap="YlGnBu")
            plt.xlabel('Prediction')
            plt.ylabel('Label')
            plt.title(key)
            i+= 1
        plt.show()
    out = np.array(accs).reshape(1, len(accs))
    return pd.DataFrame(out, columns=index)

# Class weights

In [10]:
wgt='balanced'
clfs = [LogisticRegression(solver = 'lbfgs',class_weight=wgt),
        DecisionTreeClassifier(class_weight=wgt),
        RandomForestClassifier(max_depth=4,class_weight=wgt)]
clf_names = ['LR','DT','RF']
rsts = pd.DataFrame()
for model, name in zip(clfs,clf_names):
    start = time.time()
    print(name)
    model.fit(X_train, y_train.values)
    # use the model to make predictions with the test data
    y_pred = model.predict(X_test)
    end= time.time()
    # get the evaluation metrics
    result = get_accs(y_test.values, y_pred, end-start)
    result.index = [name + '-Weights']
    rsts = rsts.append(result)
print(rsts.iloc[:,5:])

LR
DT
RF
            Overall Accuracy    G-mean   Avg_Pfm  Training Time
LR-Weights          0.488749  0.380002  0.434375          6.964
DT-Weights          0.713778  0.159283  0.436530          1.925
RF-Weights          0.525938  0.240021  0.382979          6.402


In [11]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss

In [12]:
# Train uncalibrated random forest classifier on whole train and validation
# data and evaluate on test data
clf = LogisticRegression(solver = 'lbfgs',class_weight=wgt)
clf.fit(X_train, y_train)
clf_probs = clf.predict_proba(X_test)
score = log_loss(y_test, clf_probs)
print(score)
y_pred = np.argmax(clf_probs, axis=1)
# get the evaluation metrics
get_accs(y_test, y_pred, 0)

1.2917036488226223


,Class 0,Class 1,Class 2,Class 3,Class 4,Overall Accuracy,G-mean,Avg_Pfm,Training Time
0,0.517207,0.424316,0.155249,0.336478,0.691176,0.488749,0.380002,0.434375,0.0


In [14]:
sig_clf = CalibratedClassifierCV(clf, method='isotonic', cv=3)
sig_clf.fit(X_train, y_train)
sig_clf_probs = sig_clf.predict_proba(X_test)
sig_score = log_loss(y_test, sig_clf_probs)
print(sig_score)
y_pred = np.argmax(sig_clf_probs, axis=1)
# get the evaluation metrics
get_accs(y_test, sig_clf.predict(X_test), 0)

0.5677182228395451


,Class 0,Class 1,Class 2,Class 3,Class 4,Overall Accuracy,G-mean,Avg_Pfm,Training Time
0,0.999842,0.0,0.0,0.003145,0.382353,0.834978,0.016443,0.425711,0.0


In [13]:
score, sig_score

(1.2894828841522397, 0.5702825376767013)

In [15]:
y_pred = np.argmax(clf_probs, axis=1)
# get the evaluation metrics
get_accs(y_test, y_pred, 0)

,Class 0,Class 1,Class 2,Class 3,Class 4,Overall Accuracy,G-mean,Avg_Pfm,Training Time
0,0.517049,0.401367,0.167956,0.371069,0.573529,0.486827,0.375021,0.430924,0.0


## MLP

In [15]:
# Add weights
weights = len(y) / (5 * np.bincount(y))
cls_wgt = dict(zip(np.arange(5), weights))
cls_wgt

{0: 0.23969465648854962,
 1: 1.8549946294307196,
 2: 4.199160035366932,
 3: 23.89559748427673,
 4: 111.74705882352941}

In [16]:
es = EarlyStopping(monitor='val_cohen_kappa',
                   verbose=1, patience=10, mode='max',
                   restore_best_weights=True)

In [17]:
def create_mlp():
    MLP = Sequential([Dense(10,activation='relu',
                           input_dim=X_train.shape[1]),
                      Dropout(0.5),
                      Dense(5, activation='softmax')])
    MLP.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[CohenKappa(num_classes=5,sparse_labels=True)])
    return MLP

In [20]:
model = create_mlp()
start = time.time()
monitor = model.fit(X_train, y_train.values,
                    callbacks=[es],
                    validation_data=(X_val, y_val.values),
                    verbose=0, epochs=50)
end = time.time()
# use the model to make predictions with the test data
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
# get the evaluation metrics
result = get_accs(y_test.values, y_pred, end-start)
result.index = ['MLP-Weights']
rsts = rsts.append(result)
print(rsts.iloc[:,5:])

Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
             Overall Accuracy    G-mean   Avg_Pfm  Training Time
LR-Weights           0.502448  0.389768  0.446108          1.574
DT-Weights           0.425804  0.260147  0.342975          0.657
RF-Weights           0.542244  0.262294  0.402269          1.789
MLP-Weights          0.008370  0.013192  0.010781         41.764
MLP-Weights          0.521714  0.322299  0.422006         22.184
MLP-Weights          0.534137  0.363847  0.448992         23.193


In [ ]:

# MLP for Pima Indians Dataset with grid search via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy

# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer=init, activation='relu'))
	model.add(Dense(8, kernel_initializer=init, activation='relu'))
	model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
# MLP for Pima Indians Dataset with grid search via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy
 
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=8, kernel_initializer=init, activation='relu'))
	model.add(Dense(8, kernel_initializer=init, activation='relu'))
	model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
	return model
 
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

# Normal Resampling

In [15]:
Resamples = {'ROS':             RandomOverSampler(), 
             'SMOTE':           SMOTE(random_state=42),
             'BorderlineSMOTE': BorderlineSMOTE(),
             'RUS': RandomUnderSampler(), 
             'NearMiss': NearMiss()
             }
for key,value in Resamples.items():
    start = time.time()
    X_res, y_res = value.fit_resample(X_train, y_train)
    end = time.time()
    res_time = end - start
    print(key)
    print('Resampled dataset shape %s' % Counter(y_res))
    print('resampling time is {0:.2f} seconds'.format(res_time))
    
    #Logistic model
    LR = LogisticRegression(solver = 'lbfgs')
    start = time.time()
    LR.fit(X_res, y_res)
    end= time.time()
    # get the evaluation metrics
    # use the model to make predictions with the test data
    y_pred = LR.predict(X_test)
    result = get_accs(y_test.values,y_pred, tr_time= end-start)
    result['Resample time'] = res_time
    result.index = ['LR-' + key]
    rsts = rsts.append(result)
    
print(rsts.iloc[:,5:])

ROS
Resampled dataset shape Counter({1: 101446, 0: 101446, 2: 101446, 3: 101446, 4: 101446})
resampling time is 0.11 seconds
SMOTE
Resampled dataset shape Counter({1: 101446, 0: 101446, 2: 101446, 3: 101446, 4: 101446})
resampling time is 14.73 seconds
BorderlineSMOTE
Resampled dataset shape Counter({1: 101446, 0: 101446, 2: 101446, 3: 101446, 4: 101446})
resampling time is 315.54 seconds
RUS
Resampled dataset shape Counter({0: 217, 1: 217, 2: 217, 3: 217, 4: 217})
resampling time is 0.03 seconds
NearMiss
Resampled dataset shape Counter({0: 217, 1: 217, 2: 217, 3: 217, 4: 217})
resampling time is 2.04 seconds
                    Overall Accuracy    G-mean   Avg_Pfm  Training Time  \
LR-Weights                  0.487538  0.384926  0.436232          5.019   
DT-Weights                  0.716700  0.148694  0.432697          1.564   
RF-Weights                  0.567706  0.196124  0.381915          4.073   
MLP-Weights                 0.486353  0.346576  0.416465        113.833   
LR-ROS  

In [16]:
#rsts = rsts.iloc[:9,:]
#rsts

# Hybrid Resampling


In [17]:
y_dict = Counter(y_train)
y_dict

Counter({1: 13109, 0: 101446, 2: 5790, 3: 1018, 4: 217})

## Oversampling and then undersampling

In [18]:
ss = {}
for i in range(2,5):
    ss[i] = y_dict[1]
oses = ['ROS','SMOTE','BorderlineSMOTE']
uses = ['RUS','NearMiss']

In [19]:
for os_name in oses:
    for us_name in uses:
        print(os_name,'-',us_name)
        if os_name == 'ROS': 
            res = RandomOverSampler(sampling_strategy=ss)
        elif os_name == 'BorderlineSMOTE': 
            res = BorderlineSMOTE(sampling_strategy=ss)
        else: res = SMOTE(sampling_strategy=ss)
        start = time.time()
        X_res, y_res = res.fit_resample(X_train, y_train)

        if us_name == 'RUS': 
            res = RandomUnderSampler(sampling_strategy='majority')
        else: res = NearMiss(sampling_strategy='majority')  
        X_res, y_res = res.fit_resample(X_res, y_res)
        end = time.time()
        res_time = end-start

        print('Resampled dataset shape %s' % Counter(y_res))
        print('Resamling time %.2f sec' % (res_time))
        LR = LogisticRegression(solver = 'lbfgs')
        start = time.time()
        LR.fit(X_res, y_res)
        end= time.time()
        # get the evaluation metrics
        # use the model to make predictions with the test data
        y_pred = LR.predict(X_test)
        result = get_accs(y_test.values,y_pred, tr_time= end-start)
        result['Resample time'] = res_time
        result.index = ['LR-' + os_name + '-' + us_name]
        rsts = rsts.append(result)

print(rsts.iloc[:,5:])

ROS - RUS
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 0.13 sec
ROS - NearMiss
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 86.98 sec
SMOTE - RUS
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 2.64 sec
SMOTE - NearMiss
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 89.48 sec
BorderlineSMOTE - RUS
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 150.43 sec
BorderlineSMOTE - NearMiss
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 237.54 sec


,Class 0,Class 1,Class 2,Class 3,Class 4,Overall Accuracy,G-mean,Avg_Pfm,Training Time,Resample time
LR-Weights,0.514462,0.430664,0.160773,0.358491,0.661765,0.487538,0.384926,0.436232,5.019,NaN
DT-Weights,0.835315,0.138184,0.087293,0.037736,0.191176,0.716700,0.148694,0.432697,1.564,NaN
RF-Weights,0.628174,0.367920,0.006630,0.286164,0.661765,0.567706,0.196124,0.381915,4.073,NaN
MLP-Weights,0.518311,0.416016,0.105525,0.339623,0.647059,0.486353,0.346576,0.416465,113.833,NaN
LR-ROS,0.516134,0.436523,0.149724,0.349057,0.647059,0.488933,0.377034,0.432983,21.084,0.114530
LR-SMOTE,0.515156,0.419434,0.150829,0.364780,0.602941,0.486380,0.372459,0.429419,20.791,14.726981
LR-BorderlineSMOTE,0.572091,0.420166,0.227624,0.295597,0.470588,0.536807,0.376955,0.456881,22.433,315.539367
LR-RUS,0.443365,0.297607,0.262431,0.361635,0.661765,0.418739,0.383424,0.401081,0.095,0.032193
LR-NearMiss,0.012112,0.079346,0.084530,0.411950,0.808824,0.027583,0.122037,0.074810,0.094,2.035056
LR-ROS-RUS,0.513548,0.432861,0.149724,0.361635,0.632353,0.486459,0.376956,0.431707,2.617,0.133523


## Under sampling and then over sampling

In [20]:
for us_name in uses:
    for os_name in oses:
        print(us_name,'-',os_name)
        if us_name == 'RUS': 
            res = RandomUnderSampler(sampling_strategy={0: y_dict[1]})
        else: res = NearMiss(sampling_strategy={0: y_dict[1]})
        start = time.time()
        X_res, y_res = res.fit_resample(X_train, y_train)

        if os_name == 'ROS': 
            res = RandomOverSampler(sampling_strategy='not majority')
        elif os_name == 'BorderlineSMOTE': 
            res = BorderlineSMOTE(sampling_strategy='not majority')
        else: res = SMOTE(sampling_strategy='not majority')
        X_res, y_res = res.fit_resample(X_res, y_res)
        end = time.time()
        res_time = end-start
        print('Resampled dataset shape %s' % Counter(y_res))
        print('Resamling time %.2f sec' % (res_time))
        LR = LogisticRegression(solver = 'lbfgs',
                                #class_weight= 'balanced'
                                )
        start = time.time()
        LR.fit(X_res, y_res)
        end= time.time()
        # get the evaluation metrics
        # use the model to make predictions with the test data
        y_pred = LR.predict(X_test)
        result = get_accs(y_test.values,y_pred, tr_time= end-start)
        result['Resample time'] = res_time
        result.index = ['LR-' + us_name + '-' + os_name]
        rsts = rsts.append(result)

print(rsts.iloc[:,5:])

RUS - ROS
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 0.05 sec
RUS - SMOTE
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 2.62 sec
RUS - BorderlineSMOTE
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 24.61 sec
NearMiss - ROS
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 1.71 sec
NearMiss - SMOTE
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 4.27 sec
NearMiss - BorderlineSMOTE
Resampled dataset shape Counter({0: 13109, 1: 13109, 2: 13109, 3: 13109, 4: 13109})
Resamling time 25.52 sec


,Class 0,Class 1,Class 2,Class 3,Class 4,Overall Accuracy,G-mean,Avg_Pfm,Training Time,Resample time
LR-Weights,0.514462,0.430664,0.160773,0.358491,0.661765,0.487538,0.384926,0.436232,5.019,NaN
DT-Weights,0.835315,0.138184,0.087293,0.037736,0.191176,0.716700,0.148694,0.432697,1.564,NaN
RF-Weights,0.628174,0.367920,0.006630,0.286164,0.661765,0.567706,0.196124,0.381915,4.073,NaN
MLP-Weights,0.518311,0.416016,0.105525,0.339623,0.647059,0.486353,0.346576,0.416465,113.833,NaN
LR-ROS,0.516134,0.436523,0.149724,0.349057,0.647059,0.488933,0.377034,0.432983,21.084,0.114530
LR-SMOTE,0.515156,0.419434,0.150829,0.364780,0.602941,0.486380,0.372459,0.429419,20.791,14.726981
LR-BorderlineSMOTE,0.572091,0.420166,0.227624,0.295597,0.470588,0.536807,0.376955,0.456881,22.433,315.539367
LR-RUS,0.443365,0.297607,0.262431,0.361635,0.661765,0.418739,0.383424,0.401081,0.095,0.032193
LR-NearMiss,0.012112,0.079346,0.084530,0.411950,0.808824,0.027583,0.122037,0.074810,0.094,2.035056
LR-ROS-RUS,0.513548,0.432861,0.149724,0.361635,0.632353,0.486459,0.376956,0.431707,2.617,0.133523


### Refining result
NearMiss not only increased the computing time but also reduced the accuracy significantly
Apply 5-fold cross validation


In [ ]:
# Border → RUS
bor2rus = pd.DataFrame()
for i in range(10):
    print(i)
    start = time.time()
    res = BorderlineSMOTE(sampling_strategy=ss)
    print('sampling #1 ...')
    X_res, y_res = res.fit_resample(X_train, y_train)
    res = RandomUnderSampler(sampling_strategy='majority')
    print('sampling #2 ...')
    X_res, y_res = res.fit_resample(X_res, y_res)
    end = time.time()
    res_time = end-start
    # training and prediction
    start = time.time()
    LR = LogisticRegression(solver = 'lbfgs',
                            #class_weight= 'balanced'
                            )
    start = time.time()
    print('training...')
    LR.fit(X_res, y_res)
    end= time.time()
    # get the evaluation metrics
    # use the model to make predictions with the test data
    y_pred = LR.predict(X_test)
    result = get_accs(y_test.values,y_pred, tr_time= end-start)
    bor2rus = bor2rus.append(result)
bor2rus

0
sampling #1 ...
sampling #2 ...
training...
1
sampling #1 ...
sampling #2 ...
training...
2
sampling #1 ...
sampling #2 ...
training...
3
sampling #1 ...
sampling #2 ...
training...
4
sampling #1 ...
sampling #2 ...
training...
5
sampling #1 ...
sampling #2 ...
training...
6
sampling #1 ...


In [ ]:
bor2rus.describe().iloc[1:3,5:]

In [ ]:
# RUS → Borderline
rus2bor = pd.DataFrame()
for i in range(10):
    print(i)
    start = time.time()
    res = RandomUnderSampler(sampling_strategy={0: y_dict[1]})
    print('sampling #1 ...')
    X_res, y_res = res.fit_resample(X_train, y_train)
    res = BorderlineSMOTE(sampling_strategy='not majority')
    print('sampling #2 ...')
    X_res, y_res = res.fit_resample(X_res, y_res)
    end = time.time()
    res_time = end-start
    # training and prediction
    start = time.time()
    LR = LogisticRegression(solver = 'lbfgs',
                            #class_weight= 'balanced'
                            )
    start = time.time()
    print('training...')
    LR.fit(X_res, y_res)
    end= time.time()
    # get the evaluation metrics
    # use the model to make predictions with the test data
    y_pred = LR.predict(X_test)
    result = get_accs(y_test.values,y_pred, tr_time= end-start)
    rus2bor = rus2bor.append(result)
rus2bor

In [ ]:
print(rus2bor.describe().iloc[1:3,5:])
print(bor2rus.describe().iloc[1:3,5:])

In [ ]:
y_dict

In [ ]:
start = time.time()
res = RandomUnderSampler(sampling_strategy={0: y_dict[1]})
print('sampling #1 ...')
X_res, y_res = res.fit_resample(X_train, y_train)
res = BorderlineSMOTE(sampling_strategy='not majority')
print('sampling #2 ...')
X_res, y_res = res.fit_resample(X_res, y_res)
print(Counter(y_res))
end = time.time()
res_time = end-start
model = create_mlp()
start = time.time()
model.fit(X_res, y_res,
                    callbacks=[es],
                 #   class_weight = cls_wgt,
                    validation_data=(X_val, y_val.values),
                    verbose=1, epochs=50)
end = time.time()
# use the model to make predictions with the test data
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
# get the evaluation metrics
result = get_accs(y_test.values, y_pred, end-start)
result.index = ['MLP-RUS2BOR']
rsts = rsts.append(result)
print(rsts.iloc[:,5:])

In [ ]:
rsts.to_csv('VCA_Resamplings.csv')